In [1]:
import tweepy

%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter


consumer_key = '9SRz5HMehrEVQf2m7AoN4shrq'
consumer_secret = 'zZa9j55quKFTmTwm4PKx4B6RUn3OyCsEtVJmvqbLAX9d8K3Adu'
access_token = '2801486303-55EJTjYXUPvw5uzXmRQV8wTHDmiLh70BJoASUj9'
access_token_secret = 'aRGfnR8N4if56loNt0yhwChXBe61go8qTpEanmXV2RBRp'

In [2]:
# Thank you, yanofsky! Adapted from: https://gist.github.com/yanofsky/5436496

def get_all_tweets(screen_names):

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    
    #initialize a list to hold all the tweepy Tweets
    df = pd.DataFrame()
    
    for screen_name in screen_names:
        
        alltweets = []

        #make initial request for most recent tweets (200 is the maximum allowed count)
        new_tweets = api.user_timeline(screen_name = screen_name ,count=200)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #save the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        #keep grabbing tweets until there are no tweets left to grab
        while len(new_tweets) > 0:
            print("getting tweets before %s" % (oldest))

            #all subsiquent requests use the max_id param to prevent duplicates
            new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

            #save most recent tweets
            alltweets.extend(new_tweets)

            #update the id of the oldest tweet less one
            oldest = alltweets[-1].id - 1

            print("...%s tweets downloaded so far" % (len(alltweets)))

        #transform the tweepy tweets into a 2D array that will populate the csv	
        outtweets = [[tweet.user.screen_name, tweet.text.encode("utf-8")] for tweet in alltweets]

        df = df.append(pd.DataFrame(data=outtweets)).reset_index(drop=True)
        
    return df
    
    print('done!')

In [9]:
# Getting the most recent 3200 tweets from Donald Trump, Kanye West, Hillary Clinton, 
#  Taylor Swift
tweets = get_all_tweets(['realDonaldTrump','justinbieber','hillaryclinton','katyperry'])

#tweets = get_all_tweets(['acupofjoanne'])

getting tweets before 983824424532144127
...400 tweets downloaded so far
getting tweets before 971008891751485439
...600 tweets downloaded so far
getting tweets before 959393270144086015
...800 tweets downloaded so far
getting tweets before 948195478428102656
...1000 tweets downloaded so far
getting tweets before 936553863255031808
...1197 tweets downloaded so far
getting tweets before 927075137131896831
...1397 tweets downloaded so far
getting tweets before 920363714062274559
...1596 tweets downloaded so far
getting tweets before 913003340451188736
...1796 tweets downloaded so far
getting tweets before 905603224379559935
...1996 tweets downloaded so far
getting tweets before 896420822780444671
...2196 tweets downloaded so far
getting tweets before 887862941109940224
...2396 tweets downloaded so far
getting tweets before 878200921980891135
...2595 tweets downloaded so far
getting tweets before 865865814099939327
...2795 tweets downloaded so far
getting tweets before 851894518014238719


In [10]:
tweets[0].value_counts()

realDonaldTrump    3237
katyperry          3208
HillaryClinton     3201
justinbieber       3170
Name: 0, dtype: int64

In [11]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12816 entries, 0 to 12815
Data columns (total 2 columns):
0    12816 non-null object
1    12816 non-null object
dtypes: object(2)
memory usage: 200.3+ KB


In [12]:
tweets.columns = ['screenname','tweet']

In [13]:
tweets.head()

,screenname,tweet
0,realDonaldTrump,b'...despite already having signed a detailed ...
1,realDonaldTrump,b'...very common among celebrities and people ...
2,realDonaldTrump,"b'Mr. Cohen, an attorney, received a monthly r..."
3,realDonaldTrump,b'Our great financial team is in China trying ...
4,realDonaldTrump,"b'Ainsley Earnhardt, a truly great person, jus..."


In [14]:
tweets['tweet'] = tweets['tweet'].astype(str)


In [15]:
def text_cleaner(tweet):
    tweet = re.sub(r'http.*','',tweet)
    tweet = re.sub(r'bit/ly.*', "", tweet)
    tweet = re.sub(r'b\'', "", tweet)
    tweet = re.sub(r'b"', "", tweet)
    return tweet

In [16]:
for r in range(len(tweets['tweet'])):
    tweets['tweet'][r] = text_cleaner(tweets['tweet'][r])

#Dropping retweets    
tweets = tweets[tweets['tweet'].str.contains("RT") == False].reset_index(drop=True)
    

In [17]:
tweets.head(20)

,screenname,tweet
0,realDonaldTrump,...despite already having signed a detailed le...
1,realDonaldTrump,...very common among celebrities and people of...
2,realDonaldTrump,"Mr. Cohen, an attorney, received a monthly ret..."
3,realDonaldTrump,Our great financial team is in China trying to...
4,realDonaldTrump,"Ainsley Earnhardt, a truly great person, just ..."
5,realDonaldTrump,"As everybody is aware, the past Administration..."
6,realDonaldTrump,\xe2\x80\x9cThis isn\xe2\x80\x99t some game. Y...
7,realDonaldTrump,I have been briefed on the U.S. C-130 \xe2\x80...
8,realDonaldTrump,Congratulations @SecPompeo!
9,realDonaldTrump,A Rigged System - They don\xe2\x80\x99t want t...


In [18]:
# Concatenizing all tweets into one text and parsing.
nlp = spacy.load('en')

text = tweets['tweet'].str.cat()
text = nlp(text)

In [19]:
# Parse tweets
nlp = spacy.load('en')

parsed = []

for r in tweets['tweet']:
    p = nlp(r)
    parsed.append(p)
    
    

In [20]:
tweets['tweet'] = parsed

In [21]:
tweets.head()

,screenname,tweet
0,realDonaldTrump,"(..., despite, already, having, signed, a, det..."
1,realDonaldTrump,"(..., very, common, among, celebrities, and, p..."
2,realDonaldTrump,"(Mr., Cohen, ,, an, attorney, ,, received, a, ..."
3,realDonaldTrump,"(Our, great, financial, team, is, in, China, t..."
4,realDonaldTrump,"(Ainsley, Earnhardt, ,, a, truly, great, perso..."


## Bag of Words
Let's use bag of words! For each tweet, we will count how many times each word appears and use those counts as features.  
Maybe we can distinguish between the politicians (Trump, Clinton and Barack) and the celebrities (Kanye, Taylor Swift, and Ellen Degeneres.)

In [22]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(1000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(tweets, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = tweets['tweet']
    df['text_source'] = tweets['screenname']
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df


In [23]:
# Finding the top 2000 common words in all the tweets.

common_words = bag_of_words(text)

In [ ]:
common_words

['-PRON-',
 'be',
 'amp',
 'great',
 'trump',
 "'s",
 'thank',
 'the',
 'not',
 'good',
 'people',
 'time',
 'day',
 'today',
 'go',
 'hillary',
 'president',
 'get',
 'love',
 ' ',
 'want',
 'year',
 'america',
 'come',
 'u',
 'work',
 'new',
 'vote',
 'donald',
 'big',
 'country',
 't',
 'tax',
 'like',
 'news',
 'make',
 'know',
 'need',
 'job',
 'say',
 'woman',
 'right',
 'tonight',
 'look',
 'this',
 'let',
 'fake',
 'family',
 'american',
 'help',
 'see',
 'a',
 'live',
 'election',
 'honor',
 'thing',
 'have',
 'night',
 'will',
 'watch',
 'take',
 'u.s.',
 'man',
 'do',
 '\\xe2\\x80\\x94hillary',
 'way',
 'win',
 'pay',
 'to',
 'world',
 'talk',
 'believe',
 'campaign',
 'state',
 'think',
 'if',
 'in',
 'purpose',
 'democrats',
 'life',
 'what',
 'tomorrow',
 'stand',
 'million',
 'hard',
 'house',
 'join',
 'united',
 'happy',
 'ready',
 'plan',
 'mean',
 'friend',
 'high',
 'strong',
 'cut',
 'clinton',
 'bad',
 'story',
 'support',
 'start',
 'give',
 'so',
 'national',
 '

In [ ]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(tweets, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000
Processing row 6500
